In [23]:
import sys
sys.path.append('../src')

import matplotlib.pyplot as plt
import pickle as pkl
import torch as t

from fcn import FCN
from utils import accumulate_grads_over_epochs , weights_reset

In [ ]:
f = FCN(5, 1)
for param in f.parameters():
    print(param)

weights_reset(f, [[1, 3], [0]])

In [ ]:
for param in f.parameters():
    print(param)

In [39]:
paradigms = ['sgd', 'adam']
batches = [128, 0]
device = t.device("cuda:{}".format(2))
components = [1, 5, 101]

for paradigm in paradigms:
    for batch in batches:
        for c in components:
            file = '../ckpts/mnist_centralized/history/clf_fcn_noise_None' \
                '_paradigm_{}_lr_0.01_decay_1e-05_batch_{}.pkl'.format(paradigm, batch)
            x_ax, acc_train, acc_test, l_train, l_test, grad = pkl.load(open(file, 'rb'))        
            grad = accumulate_grads_over_epochs(grad, device)
            grad0 = t.stack([_[0].flatten() for _ in grad], dim=0).T
            grad1 = t.stack([_[1].flatten() for _ in grad], dim=0).T
            print(grad0.size(), grad1.size())

            if c==101:
                pca = PCA()
            else:
                pca = PCA(n_components=c)
            grad0 = pca.fit_transform(grad0.cpu().numpy())
            print(sum(pca.explained_variance_ratio_[:c]))
            grad1 = pca.fit_transform(grad1.cpu().numpy())
            print(sum(pca.explained_variance_ratio_[:c]))
            grad0 = t.Tensor(grad0).T.to(device)
            grad1 = t.Tensor(grad1).T.to(device)
            print(grad0.size(), grad1.size())
            file = '../ckpts/mnist_centralized/data/grad'\
                '_pca_paradigm_{}_batch_{}_components_{}.pkl'.format(
                paradigm, batch, c)
            print('Saving: {}'.format(file))
            pkl.dump((grad0, grad1), open(file, 'wb'))

torch.Size([7840, 101]) torch.Size([10, 101])
0.8628233671188354
0.6535676717758179
torch.Size([1, 7840]) torch.Size([1, 10])
Saving: ../ckpts/mnist_centralized/data/grad_pca_paradigm_sgd_batch_128_components_1.pkl
torch.Size([7840, 101]) torch.Size([10, 101])
0.9662916837260127
0.9481319785118103
torch.Size([5, 7840]) torch.Size([5, 10])
Saving: ../ckpts/mnist_centralized/data/grad_pca_paradigm_sgd_batch_128_components_5.pkl
torch.Size([7840, 101]) torch.Size([10, 101])
0.9999999229426066
1.0000000605359705
torch.Size([101, 7840]) torch.Size([10, 10])
Saving: ../ckpts/mnist_centralized/data/grad_pca_paradigm_sgd_batch_128_components_101.pkl
torch.Size([7840, 101]) torch.Size([10, 101])
0.9558389186859131
0.5722149014472961
torch.Size([1, 7840]) torch.Size([1, 10])
Saving: ../ckpts/mnist_centralized/data/grad_pca_paradigm_sgd_batch_0_components_1.pkl
torch.Size([7840, 101]) torch.Size([10, 101])
0.9999844329649932
0.9999303127988242
torch.Size([5, 7840]) torch.Size([5, 10])
Saving: ../